The purpose of this exercise is to demonstrate the use of RNNs using generated data. By generating data ourselves, we have more control over what we train and test the model on.


In [64]:
import keras as K
import numpy as np
import pandas as pd

from random import randint, random
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.recurrent import LSTM, SimpleRNN

from sklearn.metrics import confusion_matrix, f1_score

# Types of Cats

Cats are lazy. They do only one thing every hour. There exists several types of cats. 

Testing that RNN's are capable of performing sequence prediction over different kinds of sequences.

In [4]:
# Most cats perform one of four actions
EAT = 0
NAP = 1
SCRATCH = 2 
BEAT_UP_STINKY_DOG = 3

NUM_POSSIBLE_NORMAL_ACTIONS = 4
NUM_POSSIBLE_ACTIONS = 5

In [5]:
def next_action_round_robin(current_action): 
    return (current_action + 1) % NUM_POSSIBLE_NORMAL_ACTIONS

def random_action(): 
    return randint(0, NUM_POSSIBLE_NORMAL_ACTIONS - 1)

In [6]:
# Predictable Cat behaves in a round robin fashion. Robin... how ironic. 
def predictable_cat():
    action = random_action()
    while True: 
        yield action
        action = next_action_round_robin(action)

In [7]:
# OCD Cat does everything in threes, no more, no less...
def ocd_cat(): 
    action = random_action()
    while True: 
        yield action
        yield action
        yield action
        action = next_action_round_robin(action)

In [8]:
# Unpredictable cat performs a random action sometimes, and resumes the sequence from there
# This adds some noise to our data, the RNN seems robust to that based on accuracy score. 
# On hindsight, should have tagged the cats to check which ones get misclassified.
def unpredictable_cat():
    action = random_action()
    while True:
        action = next_action_round_robin(action) if random() < 0.8 else random_action()
        yield action

In [9]:
# Smelly Cat raids the dumpster every other action
RAID_DUMPSTER = 4

def smelly_cat():
    action = random_action()
    while True:
        yield action
        yield RAID_DUMPSTER
        action = next_action_round_robin(action)

In [13]:
# Let's have a look at Smelly Cat's day
cat = smelly_cat()
smelly_actions = [next(cat) for i in range(0,20)]
print(smelly_actions)

[0, 4, 1, 4, 2, 4, 3, 4, 0, 4, 1, 4, 2, 4, 3, 4, 0, 4, 1, 4]


In [14]:
def cat_randomizer():
#     cat_gens = [predictable_cat, ocd_cat, smelly_cat]
    cat_gens = [predictable_cat, unpredictable_cat, ocd_cat, smelly_cat]

    while True:
        yield cat_gens[randint(0, 2)]()

In [15]:
def actions_for(cat, num_actions):
    return [next(cat) for i in range(0, num_actions)]

# Data Preparation

We will use one-hot encoding to represent actions.

In [16]:
def one_hot(labels, num_classes): 
    return [[1 if i == label else 0 for i in range(0, num_classes)] for label in labels]

In [17]:
one_hot(smelly_actions, NUM_POSSIBLE_ACTIONS)

[[1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1]]

And now for some kitty action!

In [18]:
def bring_me_some_cats(cat_count):
    rand_cat = cat_randomizer()
    for i in range(0, cat_count):
        yield next(rand_cat)
#         yield predictable_cat()

In [20]:
num_actions = 51
num_cats = 500
kitties_actions = [one_hot(actions_for(cat, num_actions), NUM_POSSIBLE_ACTIONS) for cat in bring_me_some_cats(num_cats)]
kitties_actions

[[[0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0]],
 [[0, 0, 1, 0, 0],
  [0, 0, 1,

Splice out the last action, this is the "label" that we want to predict.

In [21]:
X = kitties_initial_actions = [k[:num_actions - 1] for k in kitties_actions]
y = kitties_last_action = [k[-1] for k in kitties_actions]

In [47]:
y[1]

[0, 1, 0, 0, 0]

Split the data set into a training set and test set.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

# Construct Model

## Model Hyperparameters

In [56]:
EPOCHS = 120
HIDDEN = 10

In [65]:
model = Sequential()
# model.add(LSTM(HIDDEN, input_shape=[50, 5]))
model.add(SimpleRNN(HIDDEN, input_shape=[50, 5]))
model.add(Dense(5, activation='softmax'))

#model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [66]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=100)

Epoch 1/120
187/187 [==============================] - 0s - loss: 1.7243 - acc: 0.0000e+00     
Epoch 2/120
187/187 [==============================] - 0s - loss: 1.6857 - acc: 0.0000e+00     
Epoch 3/120
187/187 [==============================] - 0s - loss: 1.6491 - acc: 0.0000e+00     
Epoch 4/120
187/187 [==============================] - 0s - loss: 1.6158 - acc: 0.0000e+00     
Epoch 5/120
187/187 [==============================] - 0s - loss: 1.5841 - acc: 0.0000e+00     
Epoch 6/120
187/187 [==============================] - 0s - loss: 1.5553 - acc: 0.0160         
Epoch 7/120
187/187 [==============================] - 0s - loss: 1.5288 - acc: 0.0160     
Epoch 8/120
187/187 [==============================] - 0s - loss: 1.5036 - acc: 0.0160     
Epoch 9/120
187/187 [==============================] - 0s - loss: 1.4802 - acc: 0.0160     
Epoch 10/120
187/187 [==============================] - 0s - loss: 1.4579 - acc: 0.1016     
Epoch 11/120
187/187 [==============================] -

187/187 [==============================] - 0s - loss: 0.4545 - acc: 1.0000     
Epoch 89/120
187/187 [==============================] - 0s - loss: 0.4467 - acc: 1.0000     
Epoch 90/120
187/187 [==============================] - 0s - loss: 0.4391 - acc: 1.0000     
Epoch 91/120
187/187 [==============================] - 0s - loss: 0.4315 - acc: 1.0000     
Epoch 92/120
187/187 [==============================] - 0s - loss: 0.4241 - acc: 1.0000     
Epoch 93/120
187/187 [==============================] - 0s - loss: 0.4169 - acc: 1.0000     
Epoch 94/120
187/187 [==============================] - 0s - loss: 0.4098 - acc: 1.0000     
Epoch 95/120
187/187 [==============================] - 0s - loss: 0.4029 - acc: 1.0000     
Epoch 96/120
187/187 [==============================] - 0s - loss: 0.3960 - acc: 1.0000     
Epoch 97/120
187/187 [==============================] - 0s - loss: 0.3894 - acc: 1.0000     
Epoch 98/120
187/187 [==============================] - 0s - loss: 0.3828 - acc: 1.

# Model Evaluation

In [29]:
def label_from_vector(v): 
    return np.argmax(v)

In [33]:
y_predicted = model.predict(X_test)
y_predicted[:3]

array([[ 0.02603357,  0.13177615,  0.78390849,  0.03979288,  0.01848896],
       [ 0.74025142,  0.11093298,  0.00875915,  0.09585754,  0.04419891],
       [ 0.0040393 ,  0.03468275,  0.87370211,  0.07283571,  0.01474009]], dtype=float32)

In [34]:
actual_labels = [label_from_vector(yt) for yt in y_test]
predicted_labels = [label_from_vector(yp) for yp in y_predicted]

In [35]:
confusion_matrix(actual_labels, predicted_labels)

array([[23,  1,  2,  0],
       [ 1, 32,  3,  1],
       [ 0,  3, 31,  0],
       [ 1,  0,  0, 27]])

In [36]:
f1_score(actual_labels, predicted_labels, average='macro')

0.90716827827021229

# Sometimes, cats' behaviors depend on the weather

The purpose of this section is (still unrealized) two-fold. 
1) Show the effectiveness on LSTMs on tasks that RNN's cannot handle (long sequences where associations are separated far in time),
2) Incorporate an input (weather) at each time step and in combination with past history, make a prediction.

Note: I have tried varying the data some ways, but have not yet found a case where LSTMs outperform simple RNN's. 

In [37]:
SUNNY = 0
RAINY = 1    

In [38]:
def sunny_rainy_sunny(rainy_period_start, rainy_duration):
    for i in range(0, rainy_period_start):
        yield SUNNY
    for i in range(0, rainy_duration):
        yield RAINY
    while True: 
        yield SUNNY

In [39]:
def rain_cat(weather):
    action = random_action()
    while True:
        if next(weather) == RAINY: 
            yield NAP
        else:
            yield action
            action = next_action_round_robin(action)

In [40]:
import functools

def random_weather_sequence():
    srs = sunny_rainy_sunny(randint(5, 15), randint(20, 30))
    return srs

rain_cats = [rain_cat(random_weather_sequence()) for i in range(250)]
rain_cats_actions = [one_hot(actions_for(cat, num_actions), NUM_POSSIBLE_ACTIONS) for cat in rain_cats]

In [41]:
rain_cats_actions

[[[0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0]],
 [[1, 0, 0, 0, 0],
  [0, 1, 0,

In [42]:
X = rain_cats_initial_actions = [k[:num_actions - 1] for k in rain_cats_actions]
y = rain_cats_last_action = [k[-1] for k in rain_cats_actions]

Split the data set into a training set and test set.

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [44]:
%pprint
print(X_train)
# %pprint

Pretty printing has been turned OFF
[[[0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0]], [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0

# Construct Model

## Model Hyperparameters

In [48]:
EPOCHS = 120
HIDDEN = 10

In [49]:
model = Sequential()
# model.add(LSTM(HIDDEN, input_shape=[50, 5]))
model.add(SimpleRNN(HIDDEN, input_shape=[50, 5]))
model.add(Dense(5, activation='softmax'))

#model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=100)

Epoch 1/120
187/187 [==============================] - 0s - loss: 1.5414 - acc: 0.2888     
Epoch 2/120
187/187 [==============================] - 0s - loss: 1.5189 - acc: 0.3797     
Epoch 3/120
187/187 [==============================] - 0s - loss: 1.4965 - acc: 0.4599     
Epoch 4/120
187/187 [==============================] - 0s - loss: 1.4743 - acc: 0.4599     
Epoch 5/120
187/187 [==============================] - 0s - loss: 1.4518 - acc: 0.4652     
Epoch 6/120
187/187 [==============================] - 0s - loss: 1.4301 - acc: 0.4652     
Epoch 7/120
187/187 [==============================] - 0s - loss: 1.4080 - acc: 0.4920     
Epoch 8/120
187/187 [==============================] - 0s - loss: 1.3865 - acc: 0.4973     
Epoch 9/120
187/187 [==============================] - 0s - loss: 1.3651 - acc: 0.5775     
Epoch 10/120
187/187 [==============================] - 0s - loss: 1.3442 - acc: 0.7326     
Epoch 11/120
187/187 [==============================] - 0s - loss: 1.3236 - acc

# Model Evaluation

In [51]:
def label_from_vector(v): 
    return np.argmax(v)

In [52]:
y_predicted = model.predict(X_test)

In [53]:
actual_labels = [label_from_vector(yt) for yt in y_test]
predicted_labels = [label_from_vector(yp) for yp in y_predicted]

In [54]:
confusion_matrix(actual_labels, predicted_labels)

array([[22,  0,  0,  0],
       [ 0, 16,  0,  0],
       [ 0,  0,  7,  0],
       [ 0,  0,  0, 18]])

In [55]:
f1_score(actual_labels, predicted_labels, average='macro')

1.0